In [1]:
import torch
import numpy as np

torch.manual_seed(1234)

# Quick Start
https: // pytorch.org / tutorials / beginner / basics / quickstart_tutorial.html

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


/Users/philippsadler/Opts/Apps/anaconda3/envs/golmi/lib/python3.9/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-x1rp5px8/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
from torch.utils.data.dataloader import default_collate

batch_size = 64


def my_collate(data):
    # this might be necessary for sequence processing
    # when there are varying sequence lengths
    return default_collate(data)  # this is just the default


# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, collate_fn=my_collate)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=my_collate)

# Have a look at a data sample
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


# Define model
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()  # flatten the 2D input from (1,28,28) to (784)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),  # this is basically: torch.mm(x, W) + b , W=(784,512), b=(1,512)
            nn.ReLU(),  # this is an activation function
            nn.Linear(512, 512),  # Note: the linear layer does not come with an activation
            nn.ReLU(),  # this is an activation function
            nn.Linear(512, 10)  # there are 10 output units (classes)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits  # this is the "raw" output (no softmax yet)


model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# the loss function there are all kinds of other losses
# e.g. MSELoss (for regression or de-noising), NLLLoss (for classification), BCSELoss (for multi-label classification)
loss_fn = nn.CrossEntropyLoss(weight=torch.ones(10)) # all classes have the same weight here

# the optimizer, a good default is actually Adam, but the Quickstart uses SGD
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0, weight_decay=1e-6)  # add the model parameters to optimize

In [7]:
# sometimes you do not want to optimize all parameters
# then do this with them before the optimizer init
for name, param in model.named_parameters():
    print(name)
    if name == "pretrained.weight":
        param.requires_grad = False

linear_relu_stack.0.weight
linear_relu_stack.0.bias
linear_relu_stack.2.weight
linear_relu_stack.2.bias
linear_relu_stack.4.weight
linear_relu_stack.4.bias


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # setting the internal "train" mode e.g. apply dropout and batch normalization
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # to gpu if possible (data always comes from cpu)

        # Compute prediction error
        pred = model(X)  # this calls NeuralNetwork.forward(X)
        loss = loss_fn(pred, y)  # loss function applies softmax internally

        # Backpropagation
        # The order is actually important!
        optimizer.zero_grad()  # erase gradients
        loss.backward()  # compute gradients
        optimizer.step()  # update parameters

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # turn-off "features" that should be only applied during training
    # e.g. dropout and batch normalization
    model.eval()
    test_loss, correct = 0, 0
    """
    Disabling gradient calculation is useful for inference, when you are sure
    that you will not call :meth:`Tensor.backward()`. It will reduce memory
    consumption for computations that would otherwise have `requires_grad=True`.

    In this mode, the result of every computation will have
    `requires_grad=False`, even when the inputs have `requires_grad=True`.
    """
    # no gradients to be computed during test (faster inference, less memory, no accidental change of model parameters)
    # if you do call backward here, then you train on the test data!
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # argmax over the logits (same as on softmax)
            correct += (torch.argmax(pred, dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308908  [    0/60000]
loss: 2.298760  [ 6400/60000]
loss: 2.287426  [12800/60000]
loss: 2.254387  [19200/60000]
loss: 2.235636  [25600/60000]
loss: 2.230029  [32000/60000]
loss: 2.213425  [38400/60000]
loss: 2.177681  [44800/60000]
loss: 2.169705  [51200/60000]
loss: 2.167521  [57600/60000]
Test Error: 
 Accuracy: 36.9%, Avg loss: 2.154087 

Epoch 2
-------------------------------
loss: 2.185116  [    0/60000]
loss: 2.123109  [ 6400/60000]
loss: 2.093073  [12800/60000]
loss: 2.092469  [19200/60000]
loss: 2.048514  [25600/60000]
loss: 2.009566  [32000/60000]
loss: 2.037364  [38400/60000]
loss: 1.964284  [44800/60000]
loss: 1.932795  [51200/60000]
loss: 1.924343  [57600/60000]
Test Error: 
 Accuracy: 56.8%, Avg loss: 1.877862 

Epoch 3
-------------------------------
loss: 1.831652  [    0/60000]
loss: 1.857685  [ 6400/60000]
loss: 1.804704  [12800/60000]
loss: 1.813338  [19200/60000]
loss: 1.754398  [25600/60000]
loss: 1.758629  [32000/600

In [11]:
# Save checkpoints via PyTorch
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
# Load checkpoints via PyTorch
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


#### Debugging

In [19]:
x = torch.arange(9).view(3, 3)
x = x.unsqueeze(dim=1)
print(x.shape)

x = x.squeeze()
print(x.shape)

torch.Size([3, 1, 3])
torch.Size([3, 3])


In [20]:
x = torch.arange(9).view(3, 3)

print(x)
print("---")
new_x = torch.cat([x, x], dim=1)  # extends an existing dimension
print(new_x.shape)
print(new_x)

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
---
torch.Size([3, 6])
tensor([[0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5],
        [6, 7, 8, 6, 7, 8]])


Concated tensors must have the same dimensions, except in the concat dimension!

In [21]:
torch.cat([torch.randn(3, 1, 512), torch.randn(3, 2, 1024)], dim=1)

RuntimeError: Sizes of tensors must match except in dimension 1. Got 512 and 1024 in dimension 2 (The offending index is 1)

In [ ]:
torch.cat([torch.randn(3, 1, 512), torch.randn(3, 10, 512)], dim=1).shape

We can also concatenate on a new 0th dimension to "stack" the tensors:

In [ ]:
x = torch.arange(9).view(3, 3)
print(x)
print("---")
new_x = torch.stack([x, x])  # adds a new dimension
print(new_x.shape)
print(new_x)

Stacked tensors must have the same shape!

In [ ]:
torch.stack([torch.randn(3, 512), torch.randn(10, 512)], dim=0).shape

In [ ]:
torch.stack([torch.randn(3, 512), torch.randn(3, 512)], dim=0).shape

Make sure that the dimensions match when using matmul

In [ ]:
x1 = torch.randn((3, 512))
W1 = torch.randn((512, 1024))
(x1 @ W1).shape

In [ ]:
(W1.T @ x1.T).shape  # this also works but puts the batch dimension to the last (which might not be wanted)

In [ ]:
x1 = torch.randn((3, 512))
W1 = torch.randn((1024, 512))

In [ ]:
(x1 @ W1).shape

In [ ]:
(x1 @ W1.T).shape

Important Tensors can be converted to numpy only on cpu!

In [ ]:
x.numpy()  # this fails, when x is on cuda

In [ ]:
x.cpu().numpy()